In [1]:
import json
from simulator import Athlete, Simulator
from sklearn.neighbors import KernelDensity
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
from functools import partial
%matplotlib inline

In [2]:
athletes_data = json.load(open('data.json'))

In [3]:
def get_sample(func):
    return func()[0][0]

In [4]:
athletes = []
for i,j in athletes_data.items():
    vals = np.array(j ['fencing'])[:,np.newaxis]
    kde  = KernelDensity(bandwidth=0.05).fit(vals)
    fencing = partial(get_sample,kde.sample)
    vals = np.array(j ['swiming'])[:,np.newaxis]
    kde2  = KernelDensity(bandwidth=1).fit(vals)
    swiming = partial(get_sample,kde2.sample)
    vals = np.array(j ['laser-run'])[:,np.newaxis]
    kde3  = KernelDensity(bandwidth=5).fit(vals)
    laser_run = partial(get_sample,kde3.sample)
    vals = np.array(j ['riding'])[:,np.newaxis]
    kde4  = KernelDensity(bandwidth=10).fit(vals)
    riding = partial(get_sample,kde3.sample)
    athlete = Athlete(i,fencing,swiming,riding,laser_run)
    athletes.append(athlete)

In [5]:
tournament = Simulator(athletes)

In [6]:
#tournament.run()

In [7]:
#res = list(map(lambda x: x.name,tournament.winners))
#for i in tournament.winners:
#    print(i.name,i.points, i.time, i.handicapTime)

In [8]:
#tournament.reset()

In [9]:
results = {i.name:[] for i in athletes}
nsim = 10000

In [ ]:
for i in range(nsim):
    tournament.run()
    res = list(map(lambda x: x.name,tournament.winners))
    tournament.reset()
    for n,name in enumerate(res):
        results[name].append(n)

In [ ]:
confidences = {}

for name, places in results.items():
    mean = np.mean(places)
    interval = stats.t.interval(0.95, len(places)-1, loc=mean, scale=stats.sem(places))
    confidences[name]= {
        'mean':mean,
        'interval': interval,
        'gap': mean-interval[0]
    }

In [ ]:
sorted_athletes = list(sorted(confidences.items(),key=lambda x: x[1]['mean']+x[1]['gap']))

In [ ]:
fix_results = {}
min_rank = math.floor(sorted_athletes[0][1]['mean'])
for n,(name, data) in enumerate(sorted_athletes):
    rank = int(math.floor(data['mean'])-min_rank) + 1
    fix_results[name] = {
        'rank': rank,
        'gap': data['gap'],
        'sorted_rank': n,
        'nation': athletes_data[name]['nation']
    }

In [ ]:
json.dump(fix_results,open('men_pentathlon.json','w'), indent=2)

In [ ]:
sorted_athletes